<a href="https://colab.research.google.com/github/jmmiddour/DS-Unit-1-Sprint-2-Statistics/blob/master/module1/LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

### Goal 1
Load and clean the data (or determine the best method to drop observations when running tests)

In [1]:
# Import my libraries:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.stats import ttest_ind, ttest_1samp

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# The .data table is an old format but still readable, just have to add the headers.
# Get my data:
! wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

--2020-05-21 03:25:20--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data.1’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.04s   

2020-05-21 03:25:20 (476 KB/s) - ‘house-votes-84.data.1’ saved [18171/18171]



In [3]:
# Make the data into a dataframe:
headers=['Party', 'Hanicapped-Infants', 'Water-Project', 'Budget', 
         'Physician-Fee-Freeze', 'El-Salvador-Aid', 'Religious-Groups', 
         'Anti-Satellite-Bill', 'Aid-to-Contras', 'MX-Missile', 'Immigration',
         'Synfuels', 'Education', 'Right-to-Sue', 'Crime', 'Duty-Free', 
         'South-Africa']

# Since the mmissing values are coded with ?, need to include na_values to 
#   replace all the ? with NaN:
house = pd.read_csv('house-votes-84.data', header=None, names=headers, 
                    na_values='?')

# Print the first 5 rows to verify it reads in correctly:
house.head()

,Party,Hanicapped-Infants,Water-Project,Budget,Physician-Fee-Freeze,El-Salvador-Aid,Religious-Groups,Anti-Satellite-Bill,Aid-to-Contras,MX-Missile,Immigration,Synfuels,Education,Right-to-Sue,Crime,Duty-Free,South-Africa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


In [4]:
# Need to recode the y and n votes with binary values to carry out comparisons:
house = house.replace({'y':1, 'n':0})

# Check my work:
house.head()

,Party,Hanicapped-Infants,Water-Project,Budget,Physician-Fee-Freeze,El-Salvador-Aid,Religious-Groups,Anti-Satellite-Bill,Aid-to-Contras,MX-Missile,Immigration,Synfuels,Education,Right-to-Sue,Crime,Duty-Free,South-Africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [5]:
# What are the value counts of the parties?
house['Party'].value_counts()

democrat      267
republican    168
Name: Party, dtype: int64

In [0]:
# Create a new varaiable for each party:
dem = house[house['Party'] == 'democrat']
rep = house[house['Party'] == 'republican']

In [7]:
# Check my work:
print (dem)
print (rep)

        Party  Hanicapped-Infants  ...  Duty-Free  South-Africa
2    democrat                 NaN  ...        0.0           0.0
3    democrat                 0.0  ...        0.0           1.0
4    democrat                 1.0  ...        1.0           1.0
5    democrat                 0.0  ...        1.0           1.0
6    democrat                 0.0  ...        1.0           1.0
..        ...                 ...  ...        ...           ...
425  democrat                 0.0  ...        1.0           NaN
426  democrat                 1.0  ...        1.0           1.0
428  democrat                 NaN  ...        1.0           1.0
429  democrat                 1.0  ...        1.0           1.0
431  democrat                 0.0  ...        0.0           1.0

[267 rows x 17 columns]
          Party  Hanicapped-Infants  ...  Duty-Free  South-Africa
0    republican                 0.0  ...        0.0           1.0
1    republican                 0.0  ...        0.0           NaN
7    repu

In [8]:
# Check for missing values in my columns:
house.isnull().sum()

Party                     0
Hanicapped-Infants       12
Water-Project            48
Budget                   11
Physician-Fee-Freeze     11
El-Salvador-Aid          15
Religious-Groups         11
Anti-Satellite-Bill      14
Aid-to-Contras           15
MX-Missile               22
Immigration               7
Synfuels                 21
Education                31
Right-to-Sue             25
Crime                    17
Duty-Free                28
South-Africa            104
dtype: int64

### Goal 2
Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01

#### Run Tests
Running ttests on all bills that have more Democrate votes than Republican votes :

In [9]:
# Value counts and means of votes on Hanicapped-Infants:
print (dem['Hanicapped-Infants'].value_counts())
print (rep['Hanicapped-Infants'].value_counts())
print ('')
print ('Average of yes votes on Hanicapped-Infants:')
print ('Dems:', dem['Hanicapped-Infants'].mean())
print ('Reps:', rep['Hanicapped-Infants'].mean())

1.0    156
0.0    102
Name: Hanicapped-Infants, dtype: int64
0.0    134
1.0     31
Name: Hanicapped-Infants, dtype: int64

Average of yes votes on Hanicapped-Infants:
Dems: 0.6046511627906976
Reps: 0.18787878787878787


In [10]:
# Run ttest on Hanicapped-Infants:
ttest_ind(dem['Hanicapped-Infants'], rep['Hanicapped-Infants'], 
          nan_policy='omit')

Ttest_indResult(statistic=9.205264294809222, pvalue=1.613440327937243e-18)

In [11]:
# Value counts and means of votes on Anti-Satellite-Bill:
print (dem['Anti-Satellite-Bill'].value_counts())
print (rep['Anti-Satellite-Bill'].value_counts())
print ('')
print ('Average of yes votes on Anti-Satellite-Bill:')
print ('Dems:', dem['Anti-Satellite-Bill'].mean())
print ('Reps', rep['Anti-Satellite-Bill'].mean())

1.0    200
0.0     59
Name: Anti-Satellite-Bill, dtype: int64
0.0    123
1.0     39
Name: Anti-Satellite-Bill, dtype: int64

Average of yes votes on Anti-Satellite-Bill:
Dems: 0.7722007722007722
Reps 0.24074074074074073


In [12]:
# Run ttest on Anti-Satellite-Bill:
ttest_ind(dem['Anti-Satellite-Bill'], rep['Anti-Satellite-Bill'], nan_policy='omit')

Ttest_indResult(statistic=12.526187929077842, pvalue=8.521033017443867e-31)

In [13]:
# Value counts and means of votes on Aid-to-Contras:
print (dem['Aid-to-Contras'].value_counts())
print (rep['Aid-to-Contras'].value_counts())
print ('')
print ('Average of yes votes on Aid-to-Contras:')
print ('Dems:', dem['Aid-to-Contras'].mean())
print ('Reps', rep['Aid-to-Contras'].mean())

1.0    218
0.0     45
Name: Aid-to-Contras, dtype: int64
0.0    133
1.0     24
Name: Aid-to-Contras, dtype: int64

Average of yes votes on Aid-to-Contras:
Dems: 0.8288973384030418
Reps 0.15286624203821655


In [14]:
# Run ttest on Aid-to-Contras:
ttest_ind(dem['Aid-to-Contras'], rep['Aid-to-Contras'], nan_policy='omit')

Ttest_indResult(statistic=18.052093200819733, pvalue=2.82471841372357e-54)

In [15]:
# Value counts and means of votes on MX-Missile:
print (dem['MX-Missile'].value_counts())
print (rep['MX-Missile'].value_counts())
print ('')
print ('Average of yes votes on MX-Missile:')
print ('Dems:', dem['MX-Missile'].mean())
print ('Reps', rep['MX-Missile'].mean())

1.0    188
0.0     60
Name: MX-Missile, dtype: int64
0.0    146
1.0     19
Name: MX-Missile, dtype: int64

Average of yes votes on MX-Missile:
Dems: 0.7580645161290323
Reps 0.11515151515151516


In [16]:
# Run ttest on MX-Missile:
ttest_ind(dem['MX-Missile'], rep['MX-Missile'], nan_policy='omit')

Ttest_indResult(statistic=16.437503268542994, pvalue=5.03079265310811e-47)

In [17]:
# Value counts and means of votes on Synfuels:
print (dem['Synfuels'].value_counts())
print (rep['Synfuels'].value_counts())
print ('')
print ('Average of yes votes on Synfuels:')
print ('Dems:', dem['Synfuels'].mean())
print ('Reps', rep['Synfuels'].mean())

1.0    129
0.0    126
Name: Synfuels, dtype: int64
0.0    138
1.0     21
Name: Synfuels, dtype: int64

Average of yes votes on Synfuels:
Dems: 0.5058823529411764
Reps 0.1320754716981132


In [18]:
# Run ttest on Synfuels:
ttest_ind(dem['Synfuels'], rep['Synfuels'], nan_policy='omit')

Ttest_indResult(statistic=8.293603989407588, pvalue=1.5759322301054064e-15)

In [19]:
# Value counts and means of votes on Duty-Free:
print (dem['Duty-Free'].value_counts())
print (rep['Duty-Free'].value_counts())
print ('')
print ('Average of yes votes on Duty-Free:')
print ('Dems:', dem['Duty-Free'].mean())
print ('Reps', rep['Duty-Free'].mean())

1.0    160
0.0     91
Name: Duty-Free, dtype: int64
0.0    142
1.0     14
Name: Duty-Free, dtype: int64

Average of yes votes on Duty-Free:
Dems: 0.6374501992031872
Reps 0.08974358974358974


In [20]:
# Run ttest on Duty-Free:
ttest_ind(dem['Duty-Free'], rep['Duty-Free'], nan_policy='omit')

Ttest_indResult(statistic=12.853146132542978, pvalue=5.997697174347365e-32)

In [21]:
# Value counts and means of votes on South-Africa:
print (dem['South-Africa'].value_counts())
print (rep['South-Africa'].value_counts())
print ('')
print ('Average of yes votes on South-Africa:')
print ('Dems:', dem['South-Africa'].mean())
print ('Reps', rep['South-Africa'].mean())

1.0    173
0.0     12
Name: South-Africa, dtype: int64
1.0    96
0.0    50
Name: South-Africa, dtype: int64

Average of yes votes on South-Africa:
Dems: 0.9351351351351351
Reps 0.6575342465753424


In [22]:
# Run ttest on South-Africa:
ttest_ind(dem['South-Africa'], rep['South-Africa'], nan_policy='omit')

Ttest_indResult(statistic=6.849454815841208, pvalue=3.652674361672226e-11)

#### Result
The Budget bill received the Highest number of yes votes from Demoncrates and it has a pvalue < 0.01


In [23]:
# Value counts and means of votes on Budget:
print (dem['Budget'].value_counts())
print (rep['Budget'].value_counts())
print ('')
print ('Average of yes votes on Budget:')
print ('Dems:', dem['Budget'].mean())
print ('Reps', rep['Budget'].mean())

1.0    231
0.0     29
Name: Budget, dtype: int64
0.0    142
1.0     22
Name: Budget, dtype: int64

Average of yes votes on Budget:
Dems: 0.8884615384615384
Reps 0.13414634146341464


In [24]:
# Run ttest on Budget:
ttest_ind(dem['Budget'], rep['Budget'], nan_policy='omit')

Ttest_indResult(statistic=23.21277691701378, pvalue=2.0703402795404463e-77)

### Goal 3
Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01

#### Run Tests
Running ttests on all bills that have more Republican votes than Democrate votes:

In [25]:
# Value counts and means of votes on El-Salvador-Aid:
print (dem['El-Salvador-Aid'].value_counts())
print (rep['El-Salvador-Aid'].value_counts())
print ('')
print ('Average of yes votes on El-Salvador-Aid:')
print ('Dems:', dem['El-Salvador-Aid'].mean())
print ('Reps', rep['El-Salvador-Aid'].mean())

0.0    200
1.0     55
Name: El-Salvador-Aid, dtype: int64
1.0    157
0.0      8
Name: El-Salvador-Aid, dtype: int64

Average of yes votes on El-Salvador-Aid:
Dems: 0.21568627450980393
Reps 0.9515151515151515


In [26]:
# Run ttest on El-Salvador-Aid:
ttest_ind(dem['El-Salvador-Aid'], rep['El-Salvador-Aid'], nan_policy='omit')

Ttest_indResult(statistic=-21.13669261173219, pvalue=5.600520111729011e-68)

In [27]:
# Value counts and means of votes on Religious-Groups:
print (dem['Religious-Groups'].value_counts())
print (rep['Religious-Groups'].value_counts())
print ('')
print ('Average of yes votes on Religious-Groups:')
print ('Dems:', dem['Religious-Groups'].mean())
print ('Reps', rep['Religious-Groups'].mean())

0.0    135
1.0    123
Name: Religious-Groups, dtype: int64
1.0    149
0.0     17
Name: Religious-Groups, dtype: int64

Average of yes votes on Religious-Groups:
Dems: 0.47674418604651164
Reps 0.8975903614457831


In [28]:
# Run ttest on Religious-Groups:
ttest_ind(dem['Religious-Groups'], rep['Religious-Groups'], nan_policy='omit')

Ttest_indResult(statistic=-9.737575825219457, pvalue=2.3936722520597287e-20)

In [29]:
# Value counts and means of votes on Immigration:
print (dem['Immigration'].value_counts())
print (rep['Immigration'].value_counts())
print ('')
print ('Average of yes votes on Immigration:')
print ('Dems:', dem['Immigration'].mean())
print ('Reps', rep['Immigration'].mean())

0.0    139
1.0    124
Name: Immigration, dtype: int64
1.0    92
0.0    73
Name: Immigration, dtype: int64

Average of yes votes on Immigration:
Dems: 0.4714828897338403
Reps 0.5575757575757576


In [30]:
# Run ttest on Immigration:
ttest_ind(dem['Immigration'], rep['Immigration'], nan_policy='omit')

Ttest_indResult(statistic=-1.7359117329695164, pvalue=0.08330248490425066)

In [31]:
# Value counts and means of votes on Education:
print (dem['Education'].value_counts())
print (rep['Education'].value_counts())
print ('')
print ('Average of yes votes on Education:')
print ('Dems:', dem['Education'].mean())
print ('Reps', rep['Education'].mean())

0.0    213
1.0     36
Name: Education, dtype: int64
1.0    135
0.0     20
Name: Education, dtype: int64

Average of yes votes on Education:
Dems: 0.14457831325301204
Reps 0.8709677419354839


In [32]:
# Run ttest on Education:
ttest_ind(dem['Education'], rep['Education'], nan_policy='omit')

Ttest_indResult(statistic=-20.500685724563073, pvalue=1.8834203990450192e-64)

In [33]:
# Value counts and means of votes on Right-to-Sue:
print (dem['Right-to-Sue'].value_counts())
print (rep['Right-to-Sue'].value_counts())
print ('')
print ('Average of yes votes on Right-to-Sue:')
print ('Dems:', dem['Right-to-Sue'].mean())
print ('Reps', rep['Right-to-Sue'].mean())

0.0    179
1.0     73
Name: Right-to-Sue, dtype: int64
1.0    136
0.0     22
Name: Right-to-Sue, dtype: int64

Average of yes votes on Right-to-Sue:
Dems: 0.2896825396825397
Reps 0.8607594936708861


In [34]:
# Run ttest on Right-to-Sue:
ttest_ind(dem['Right-to-Sue'], rep['Right-to-Sue'], nan_policy='omit')

Ttest_indResult(statistic=-13.51064251060933, pvalue=1.2278581709672758e-34)

In [35]:
# Value counts and means of votes on Crime:
print (dem['Crime'].value_counts())
print (rep['Crime'].value_counts())
print ('')
print ('Average of yes votes on Crime:')
print ('Dems:', dem['Crime'].mean())
print ('Reps', rep['Crime'].mean())

0.0    167
1.0     90
Name: Crime, dtype: int64
1.0    158
0.0      3
Name: Crime, dtype: int64

Average of yes votes on Crime:
Dems: 0.35019455252918286
Reps 0.9813664596273292


In [36]:
# Run ttest on Crime:
ttest_ind(dem['Crime'], rep['Crime'], nan_policy='omit')

Ttest_indResult(statistic=-16.342085656197696, pvalue=9.952342705606092e-47)

#### Result
The Physician-Fee-Freeze bill received the Highest number of yes votes from Republicans and it has a pvalue < 0.01

In [37]:
# Value counts and means of votes on Physician-Fee-Freeze:
print (dem['Physician-Fee-Freeze'].value_counts())
print (rep['Physician-Fee-Freeze'].value_counts())
print ('')
print ('Average of yes votes on Physician-Fee-Freeze:')
print ('Dems:', dem['Physician-Fee-Freeze'].mean())
print ('Reps:', rep['Physician-Fee-Freeze'].mean())

0.0    245
1.0     14
Name: Physician-Fee-Freeze, dtype: int64
1.0    163
0.0      2
Name: Physician-Fee-Freeze, dtype: int64

Average of yes votes on Physician-Fee-Freeze:
Dems: 0.05405405405405406
Reps: 0.9878787878787879


In [38]:
# Run ttest on Physician-Fee-Freeze:
ttest_ind(dem['Physician-Fee-Freeze'], rep['Physician-Fee-Freeze'], nan_policy='omit')

Ttest_indResult(statistic=-49.36708157301406, pvalue=1.994262314074344e-177)

### Goal 4
Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

The Water-Project was split down the middle on the votes and has a pvalue > 0.1

In [39]:
# Value counts of 'yes' votes on Water-Project:
print (dem['Water-Project'].value_counts())
print (rep['Water-Project'].value_counts())
print ('')
print ('Average of yes votes on Water-Project:')
print ('Dems:', dem['Water-Project'].mean())
print ('Reps:', rep['Water-Project'].mean())

1.0    120
0.0    119
Name: Water-Project, dtype: int64
1.0    75
0.0    73
Name: Water-Project, dtype: int64

Average of yes votes on Water-Project:
Dems: 0.502092050209205
Reps: 0.5067567567567568


In [0]:
# Run ttest on Water-Project:
ttest_ind(dem['Water-Project'], rep['Water-Project'], nan_policy='omit')

Ttest_indResult(statistic=-0.08896538137868286, pvalue=0.9291556823993485)

## Stretch Goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Work on Performing a T-test without using Scipy in order to get "under the hood" and learn more thoroughly about this topic.
### Start with a 1-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://blog.minitab.com/hs-fs/hubfs/Imported_Blog_Media/701f9c0efa98a38fb397f3c3ec459b66.png?width=247&height=172&name=701f9c0efa98a38fb397f3c3ec459b66.png) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

 ### Then try a 2-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://lh3.googleusercontent.com/proxy/rJJ5ZOL9ZDvKOOeBihXoZDgfk7uv1YsRzSQ1Tc10RX-r2HrRpRLVqlE9CWX23csYQXcTniFwlBg3H-qR8MKJPBGnjwndqlhDX3JxoDE5Yg) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

 ### Then check your Answers using Scipy!

### First
1. Refactor your code into functions so it's easy to rerun with arbitrary variables.

2. Work on Performing a T-test without using Scipy in order to get "under the hood" and learn more thoroughly about this topic.

In [44]:
# Create a function to return the results of Value counts and means of votes:
def val_mean (col):
  while: 
    (df['col'] == dem['col'])
    print (dem['col'].value_counts())
  while:
    (df['col'] == (rep['col'])
    print (rep['col'].value_counts())
  print ('')
  print ('Average of yes votes on (col):')
  while (df['col'] == (dem['col']):
    print ('Dems:', dem['col'].mean())
  while (df['col'] == (rep['col']):
    print ('Reps:', rep['col'].mean())

SyntaxError: ignored

### 1-sample t-test

- Establish the conditions for your test 
 - [Calculate the T Statistic](https://blog.minitab.com/hs-fs/hubfs/Imported_Blog_Media/701f9c0efa98a38fb397f3c3ec459b66.png?width=247&height=172&name=701f9c0efa98a38fb397f3c3ec459b66.png) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

### 2-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://lh3.googleusercontent.com/proxy/rJJ5ZOL9ZDvKOOeBihXoZDgfk7uv1YsRzSQ1Tc10RX-r2HrRpRLVqlE9CWX23csYQXcTniFwlBg3H-qR8MKJPBGnjwndqlhDX3JxoDE5Yg) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

### Check your Answers 
Check your answers using Scipy!